In [2]:
import pandas as pd
from datetime import datetime

In [4]:
df = pd.read_csv(r"C:\Users\washi\OneDrive\Desktop\habit_failure_log.csv")
df

,id,habit,intensity,trigger,notes,date,time
0,1,scrolling social media,7,boredom during lunch break,wasted 2 hours on instagram reels,2026-01-01,13:45:22
1,2,eating junk food,5,stressed about upcoming exam,had a whole bag of chips while studying,2026-01-02,20:15:33
2,3,staying up late,8,started watching a new series,binged 5 episodes slept at 3am,2026-01-02,03:12:45
3,4,scrolling social media,6,procrastinating on assignment,should have been working on project,2026-01-03,16:30:12
4,5,skipping exercise,4,felt too tired after classes,made excuses again,2026-01-03,18:00:55
5,6,eating junk food,9,late night study session,ordered pizza at midnight,2026-01-04,23:45:18
6,7,procrastinating,8,assignment due tomorrow,cleaned my entire room instead of working,2026-01-04,14:20:30
7,8,scrolling social media,5,waiting for friend to arrive,quick check turned into 45 minutes,2026-01-05,11:05:42


In [5]:

# global list to store all failure records
failure_records = []

def load_data_from_file():
    """load existing failure records from json file"""
    global failure_records
    try:
        with open('failure_data.json', 'r') as file:
            failure_records = json.load(file)
            print("\nprevious data loaded successfully!")
    except FileNotFoundError:
        print("\nno previous data found. starting fresh!")
        failure_records = []
    except json.JSONDecodeError:
        print("\ndata file corrupted. starting with empty records.")
        failure_records = []

def save_data_to_file():
    """save all failure records to json file"""
    try:
        with open('failure_data.json', 'w') as file:
            json.dump(failure_records, file, indent=4)
            print("\ndata saved successfully!")
    except Exception as e:
        print(f"\nerror saving data: {e}")

def add_failure_record():
    """add a new failure incident to the tracker"""
    print("\n" + "="*50)
    print("log a failure (you messed up today!)")
    print("="*50)
    
    bad_habit = input("\nwhat bad habit did you fail at avoiding? ").strip()
    
    if not bad_habit:
        print("habit name cannot be empty!")
        return
    
    # get intensity level with validation
    while True:
        try:
            intensity = int(input("how bad was it? (1=minor slip, 10=total disaster): "))
            if 1 <= intensity <= 10:
                break
            else:
                print("please enter a number between 1 and 10!")
        except ValueError:
            print("invalid input! enter a number between 1 and 10.")
    
    trigger = input("what triggered this failure? ").strip()
    notes = input("any additional notes? (optional): ").strip()
    
    # create failure record dictionary
    failure_entry = {
        'id': len(failure_records) + 1,
        'habit': bad_habit.lower(),
        'intensity': intensity,
        'trigger': trigger.lower() if trigger else "unknown",
        'notes': notes,
        'date': datetime.now().strftime("%Y-%m-%d"),
        'time': datetime.now().strftime("%H:%M:%S")
    }
    
    # add to list using append method
    failure_records.append(failure_entry)
    
    print(f"\nfailure #{failure_entry['id']} logged! better luck tomorrow!")

def view_all_failures():
    """display all recorded failures"""
    if not failure_records:
        print("\ncongratulations! no failures recorded yet!")
        return
    
    print("\n" + "="*50)
    print(f"all recorded failures ({len(failure_records)} total)")
    print("="*50)
    
    for record in failure_records:
        print(f"\n--- failure #{record['id']} ---")
        print(f"bad habit: {record['habit']}")
        print(f"intensity: {record['intensity']}/10")
        print(f"trigger: {record['trigger']}")
        print(f"date/time: {record['date']} at {record['time']}")
        if record.get('notes'):
            print(f"notes: {record['notes']}")
    
    print("\n" + "="*50)

def search_failures():
    """search failures by habit name"""
    if not failure_records:
        print("\nno failures to search through!")
        return
    
    search_term = input("\nenter habit name to search: ").strip().lower()
    
    # filter using list comprehension
    found_records = [record for record in failure_records 
                     if search_term in record['habit'].lower()]
    
    if not found_records:
        print(f"\nno failures found for '{search_term}'")
    else:
        print(f"\nfound {len(found_records)} failure(s) matching '{search_term}':")
        for record in found_records:
            print(f"\n  {record['habit']} on {record['date']} - intensity: {record['intensity']}/10")
            print(f"  trigger: {record['trigger']}")

def view_statistics():
    """show failure statistics and patterns"""
    if not failure_records:
        print("\nno data available for statistics!")
        return
    
    print("\n" + "="*50)
    print("failure statistics")
    print("="*50)
    
    # count failures by habit using dictionary
    habit_count = {}
    total_intensity = 0
    
    for record in failure_records:
        habit_name = record['habit']
        # dictionary get method with default value
        habit_count[habit_name] = habit_count.get(habit_name, 0) + 1
        total_intensity += record['intensity']
    
    print(f"\ntotal failures logged: {len(failure_records)}")
    print(f"average intensity: {total_intensity / len(failure_records):.2f}/10")
    
    print("\n--- most common failures ---")
    # dictionary items method to get key-value pairs
    sorted_habits = sorted(habit_count.items(), key=lambda x: x[1], reverse=True)
    for habit, count in sorted_habits:
        print(f"  {habit}: {count} time(s)")
    
    # find worst failure using max
    worst_failure = max(failure_records, key=lambda x: x['intensity'])
    print(f"\n--- worst failure ---")
    print(f"  {worst_failure['habit']} (intensity: {worst_failure['intensity']}/10)")
    print(f"  date: {worst_failure['date']}")

def delete_failure():
    """delete a failure record by id"""
    if not failure_records:
        print("\nno failures to delete!")
        return
    
    view_all_failures()
    
    try:
        failure_id = int(input("\nenter failure id to delete: "))
        
        # search for record with matching id
        for i, record in enumerate(failure_records):
            if record['id'] == failure_id:
                deleted = failure_records.pop(i)  # list pop method
                print(f"\ndeleted: {deleted['habit']} from {deleted['date']}")
                return
        
        print(f"\nno failure found with id {failure_id}")
        
    except ValueError:
        print("\ninvalid input! please enter a valid id number.")

def display_menu():
    """display the main menu options"""
    print("\n" + "="*50)
    print("daily habit failure tracker")
    print("track what you don't want to do!")
    print("="*50)
    print("\n1. log a failure (you messed up!)")
    print("2. view all failures")
    print("3. search failures by habit")
    print("4. view statistics & patterns")
    print("5. delete a failure record")
    print("6. save & exit")
    print("\n" + "="*50)

def main():
    """main program loop"""
    print("\nwelcome to the habit failure tracker!")
    print("the app that keeps you accountable by tracking your slip-ups.\n")
    
    load_data_from_file()
    
    while True:
        display_menu()
        choice = input("\nenter your choice (1-6): ").strip()
        
        if choice == '1':
            add_failure_record()
        elif choice == '2':
            view_all_failures()
        elif choice == '3':
            search_failures()
        elif choice == '4':
            view_statistics()
        elif choice == '5':
            delete_failure()
        elif choice == '6':
            save_data_to_file()
            print("\nstay strong! see you tomorrow (hopefully with fewer failures)!")
            break
        else:
            print("\ninvalid choice! please select 1-6.")
        
        input("\npress enter to continue...")

# run the program
if __name__ == "__main__":
    main()


welcome to the habit failure tracker!
the app that keeps you accountable by tracking your slip-ups.


no previous data found. starting fresh!

daily habit failure tracker
track what you don't want to do!

1. log a failure (you messed up!)
2. view all failures
3. search failures by habit
4. view statistics & patterns
5. delete a failure record
6. save & exit




enter your choice (1-6):  1



log a failure (you messed up today!)



what bad habit did you fail at avoiding?  boredom during lunch break	
how bad was it? (1=minor slip, 10=total disaster):  1
what triggered this failure?  scrolling social media
any additional notes? (optional):  No



failure #1 logged! better luck tomorrow!


...


no failures found for '4'



press enter to continue... 5



daily habit failure tracker
track what you don't want to do!

1. log a failure (you messed up!)
2. view all failures
3. search failures by habit
4. view statistics & patterns
5. delete a failure record
6. save & exit




enter your choice (1-6):  6



error saving data: name 'json' is not defined

stay strong! see you tomorrow (hopefully with fewer failures)!
